In [1]:
!pip install pymongo

In [11]:
import pandas as pd
from PIL import Image
import io
df = pd.read_csv('C:\\Users\\mdc20\\full_labels.csv')

In [3]:
from pymongo import MongoClient
def get_db_handle(db_name, host, port, username, password):

    client = MongoClient(host=host,
                         port=int(port),
                         username=username,
                         password=password)
    db_handle = client['db_name']
    return db_handle, client
def get_db_default_handle():
    

    db_handle, client = get_db_handle(db_name='fashion_finder_db',
                                      host='mongodb+srv://cluster0.glnjpi9.mongodb.net/test',
                                      username='django_db_user',
                                      password='Ko4mNy6A5JEaST', # TODO: make secret
                                      port=27017)
    return db_handle, client

In [4]:
db_handle, client = get_db_default_handle()
db = client.fashion_finder_db
# db_handle.createCollection('LabeledFashionPiece', {})

In [38]:
db.create_collection('LabeledFashionPiece')

Collection(Database(MongoClient(host=['ac-3cypet9-shard-00-02.glnjpi9.mongodb.net:27017', 'ac-3cypet9-shard-00-01.glnjpi9.mongodb.net:27017', 'ac-3cypet9-shard-00-00.glnjpi9.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-5h9kqq-shard-0', tls=True), 'fashion_finder_db'), 'LabeledFashionPiece')

In [15]:
# Model definition
def create_mongo_document(client, data):
    filepath = data['filepath']
    try:
        im = Image.open(filepath)
        if im is not None:
            img_bytes = io.BytesIO()
            im.save(img_bytes, format='JPEG')

            fashion_piece_doc = {
                'img_data': img_bytes.getvalue(),
                'labels': eval(data['labels']),
                'hex_codes': eval(data['hex_codes']),
                'descriptor': data['descriptor'],
            }
            return fashion_piece_doc
        else:
            print("Image does not exist")
            print(data['filepath'])
            return None
    except:
        return None

# df_sample = df.sample(1)
# mongo_doc = df_sample.apply(lambda row: create_mongo_document(client, row), axis=1)
# doc = create_mongo_document(client, df_sample)

In [13]:
df = df[['folder', 'labels','hex_codes','filepath']]
df = df.rename({'folder':'descriptor'}, axis=1)

In [16]:
import json
from venv import create
# df.T.to_json()
df_sample = df

records = json.loads(df_sample.T.to_json()).values()
# records_2 = 

mongo_docs = [create_mongo_document(client, record) for record in records] 
# db.LabeledFashionPiece.insert_many(mongo_docs)
# print(mongo_docs)
# for record in records:
    # print(type(record))
    # fp_doc = create_mongo_document(client, record)
    # print(fp_doc)
    # break

In [ ]:
mongo_docs = filter(lambda x: x is not None, mongo_docs)

In [ ]:
db.LabeledFashionPiece.insert_many(records)

In [30]:
data = {
    'data' : {
        'labels': ['Striped', 'Tribal', 'Print', 'Muscle', 'Tee'],
        'hex_codes': ['E2DDDF', '8F716B', 'B9ADB3'],
        'descriptor': 'Some descriptor'
    }
}
id = db.LabeledFashionPiece.insert_one(data)
print(id.inserted_id)

6359e230564603a74d4fba98
